In [1]:
# https://huggingface.co/learn/audio-course/en/chapter4/fine-tuning
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification, TrainingArguments
import torchaudio
import torchaudio.transforms as T
import numpy as np
import os

In [2]:
# https://huggingface.co/docs/transformers/v4.37.2/en/model_doc/hubert#transformers.HubertForSequenceClassification

In [3]:
model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

sampling_rate = feature_extractor.sampling_rate

In [4]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

In [5]:
from datasets import load_dataset, Audio

dataset = load_dataset('audiofolder', data_dir='Rebetika_2sec')

Resolving data files:   0%|          | 0/337 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/49 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
print(dataset['train'][50])
print(dataset['train'].features['label'])

{'audio': {'path': '/media/maximos/9C33-6BBD/python/audio_singer_classification/Rebetika_2sec/train/Bithikotsis/Bithikotsis_sil_remov9_0.wav', 'array': array([0.02206421, 0.02392578, 0.02386475, ..., 0.50463867, 0.52047729,
       0.52774048]), 'sampling_rate': 44100}, 'label': 1}
ClassLabel(names=['Bellou', 'Bithikotsis', 'KaitiGkrey', 'Kazantzidis', 'KazantzidisOld', 'Ninou', 'Tsaousakis'], id=None)


In [7]:
# resample audio files to desired sample rate
dataset = dataset.cast_column('audio', Audio(sampling_rate=sampling_rate))

In [8]:
print(dataset['train'][50])

{'audio': {'path': '/media/maximos/9C33-6BBD/python/audio_singer_classification/Rebetika_2sec/train/Bithikotsis/Bithikotsis_sil_remov9_0.wav', 'array': array([0.01507542, 0.02441569, 0.0123475 , ..., 0.21213573, 0.3860018 ,
       0.54640865]), 'sampling_rate': 16000}, 'label': 1}


In [9]:
sample = dataset["train"][0]["audio"]
print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: 5.26e-05, Variance: 0.0264


In [10]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])
print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

inputs keys: ['input_values', 'attention_mask']
Mean: -4.35e-09, Variance: 1.0


In [11]:
max_duration = 10.0

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [12]:
dataset_encoded = dataset.map(
    preprocess_function,
    remove_columns=['audio'],
    batched=True,
    batch_size=100,
    num_proc=1,
)
dataset_encoded

Map:   0%|          | 0/337 [00:00<?, ? examples/s]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 337
    })
    test: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 49
    })
})

In [13]:
id2label_fn = dataset['train'].features['label'].int2str
id2label_fn(dataset['train'][50]['label'])

'Bithikotsis'

In [14]:
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(dataset_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label['2']

'KaitiGkrey'

In [15]:
num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model_name = model_id.split("/")[-1]
batch_size = 2
gradient_accumulation_steps = 4
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-2sec_voice",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=False,
)

In [17]:
import evaluate

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [18]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,1.782300,1.715721,0.551020
1,1.301100,1.360491,0.469388
2,1.152700,1.124751,0.551020
4,0.493900,0.714032,0.755102
5,0.376400,0.766802,0.714286
6,0.284800,0.578809,0.795918
8,0.274600,0.603812,0.795918
9,0.178200,0.559338,0.816327


TrainOutput(global_step=420, training_loss=0.7365313484555199, metrics={'train_runtime': 372.6965, 'train_samples_per_second': 9.042, 'train_steps_per_second': 1.127, 'total_flos': 1.5222751877204082e+16, 'train_loss': 0.7365313484555199, 'epoch': 9.94})